# Lecture: Complex Network Analysis

Prof. Dr. Michael Gertz

Winter Semester 2021/22

## Assignment 7 - Assortativity and Robustness

### Problem 7-3: Xalvi-Brunet and Sokolov Algorithm

Students: Felix Hausberger, Nils Krehl, Patrick Günther

In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import scipy

# 1. Xalvi-Brunet and Sokolov algorithm

In [71]:
def xalvi_brunet_sokolov_algorithm(graph, num_iterations, assortative):
    network = graph.copy()
    for i in range(num_iterations):
        links = np.array(list(network.edges))
        degrees = network.degree()
        # choose two random links
        choosen_indices = np.random.choice(range(len(links)), 2, replace=False)
        choosen_links = links[choosen_indices]
        
        # get corresponding nodes and their node degrees
        corresponding_nodes = choosen_links.flatten()
        corresponding_node_degrees = np.array([degrees[x] for x in corresponding_nodes])
        
        # sort the nodes by their degrees in descending order
        index_array = np.argsort(corresponding_node_degrees)[::-1]
        ordered_nodes = corresponding_nodes[index_array]
        ordered_nodes_degrees = corresponding_node_degrees[index_array]
        
        # remove the selected links
        network.remove_edge(choosen_links[0][0], choosen_links[0][1])
        network.remove_edge(choosen_links[1][0], choosen_links[1][1])
        
        # rewiring
        if assortative == True:
            network.add_edge(ordered_nodes[0], ordered_nodes[1])
            network.add_edge(ordered_nodes[2], ordered_nodes[3])
        else:
            network.add_edge(ordered_nodes[0], ordered_nodes[3])
            network.add_edge(ordered_nodes[1], ordered_nodes[2])
    
    return network

# 2. Create networks with Xalvi-Brunet and Sokolov algorithm

In [72]:
df_neutral_network = pd.read_csv('neutral_network.txt', delim_whitespace=True, header=None)

In [73]:
G_neutral_network = nx.Graph()
G_neutral_network.add_edges_from(df_neutral_network.itertuples(index=False))

In [74]:
G_assortative = xalvi_brunet_sokolov_algorithm(G_neutral_network, 5000, True)
G_disassortative = xalvi_brunet_sokolov_algorithm(G_neutral_network, 5000, False)

In [79]:
print("Degree Correlation Coefficient")
print("r = 0: neutral network; r < 0: disassortative network; r > 0: assortative network \n")
print("Neutral network Degree Correlation Coefficient: {}".format(nx.degree_pearson_correlation_coefficient(G_neutral_network)))
print("Assortative network Degree Correlation Coefficient: {}".format(nx.degree_pearson_correlation_coefficient(G_assortative)))
print("Disassortative network Degree Correlation Coefficient: {}".format(nx.degree_pearson_correlation_coefficient(G_disassortative)))

Degree Correlation Coefficient
r = 0: neutral network; r < 0: disassortative network; r > 0: assortative network 

Neutral network Degree Correlation Coefficient: -0.009246262701730106
Assortative network Degree Correlation Coefficient: 0.8059171023423379
Disassortative network Degree Correlation Coefficient: -0.6342320039448923


# 3. Plot giant component size

# 4. Disucssion

Discuss the results from the previous task: Which network is the most robust against random failures? Explain why this is the case.

In [85]:
# calculates nearest neighbor degree for singe nodes
def calculate_k_nn_single_node(G, node):
    neighbors = list(G.neighbors(node))
    return np.sum([G.degree(neighbor) for neighbor in neighbors]) / G.degree(node)

In [91]:
# get k_i
k_i_network_science = []
for node in list(G_network_science.nodes):
    k_i_network_science.append(calculate_k_nn_single_node(G_network_science, node))
    
k_i_blogs = []
for node in list(G_blogs.nodes):
    k_i_blogs.append(calculate_k_nn_single_node(G_blogs, node))
    
#k_i_javax = []
#for node in list(G_javax.nodes):
#    k_i_javax.append(calculate_k_nn_single_node(G_javax, node))


In [46]:
# calculates nearest neighbor degree for all nodes of degree k
def calculate_k_nn(k, deg_corr_mat_absolute):
    neighbors = deg_corr_mat_absolute[k]
    num_neighbors = np.sum(neighbors)
    
    return np.sum([k_prime * neighbors[k_prime] / num_neighbors for k_prime in range(len(neighbors))])

In [96]:
# get k_nn
k_nn_network_science = []
for k in range(len(deg_corr_mat_network_science_absolute[0])):
    k_nn_network_science.append(calculate_k_nn(k, deg_corr_mat_network_science_absolute))
    
k_nn_blogs = []
for k in range(len(deg_corr_mat_blogs_absolute[0])):
    k_nn_blogs.append(calculate_k_nn(k, deg_corr_mat_blogs_absolute))

#k_nn_javax = []
#for k in range(len(deg_corr_mat_javax_absolute[0])):
#    k_nn_javax.append(calculate_k_nn(k, deg_corr_mat_javax_absolute))

/opt/anaconda3/envs/complexnetworkanalysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


NameError: name 'deg_corr_mat_blogs_absolute' is not defined

In [47]:
calculate_k_nn(5, deg_corr_mat_network_science_absolute)

6.381632653061225

In [86]:
calculate_k_nn_single_node(G_network_science, 7)

4.0

In [93]:
deg_corr_mat_network_science[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])